## GENERAL INFORMATION

The API adress is : http://52.5.157.238:5000/

This notebook aim to test the API fonctions.

In [262]:
import numpy as np
import pandas as pd
import torch
import scipy
import pickle
from sklearn import *
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from bs4 import BeautifulSoup
import nltk
from nltk.stem import WordNetLemmatizer
import tensorflow_hub as hub


In [263]:
def preprocess_fct(title,text):
    #1: Delete html balises and lower text
    title = BeautifulSoup(title).get_text().lower()
    text = BeautifulSoup(text).get_text().lower()
    #2: Deelete english words:
    #tokenizer
    tokenizer = nltk.RegexpTokenizer(r'[a-zA_\-+#]*\.?[a-zA_\+#]+')
    tokens_list_title = tokenizer.tokenize(title)
    tokens_list_text = tokenizer.tokenize(text)
    english_stop_words=nltk.corpus.stopwords.words('English')
    clean_tokens_list_title = [word for word in tokens_list_title if word not in english_stop_words]
    clean_tokens_list_text = [word for word in tokens_list_text if word not in english_stop_words]
    #3: lemmatization:
    trans = WordNetLemmatizer()
    trans_title = [trans.lemmatize(word) for word in clean_tokens_list_title]
    trans_text = [trans.lemmatize(word) for word in clean_tokens_list_text]
    final_text = trans_title + trans_text
    print('Final text : ', final_text)

    return " ".join(final_text)

In [8]:
def use_embedding(text):
    #call the USE
    print('Loading the USE tokenizer ...')
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
    print('create the embedded matrix ...')
    embedding = embed([text])
    return pd.DataFrame(embedding.numpy())


In [264]:
def tfidf_embedding(text):

    # transformer = TfidfTransformer()
    loaded_vec = TfidfVectorizer(decode_error='replace', vocabulary=pickle.load(open("feature.pkl","rb")))
    # print('printing the Instance of TFidfVectorizer which has the loaded vocabulary from "feature.pkl" : ',loaded_vec)
    embedding = loaded_vec.fit_transform([text])
    dense_array = embedding.toarray()
    features = pickle.load(open("feature.pkl","rb"))
    doc_df = pd.DataFrame(dense_array)
    # from collections import OrderedDict
    # sorted_dict = OrderedDict(sorted(features.items(), key=lambda x: x[1]))
    # doc_df.columns = sorted_dict
    return doc_df
    # print('embedded transformed to a dense array :', dense_array)
    #
    # # df = pd.DataFrame(embedding[index_number].toarray())
    # # df.columns = features_name_list
    # df = pd.DataFrame(embedding)
    # features = pickle.load(open("feature.pkl","rb"))
    # # print('features : ', features)
    # print('feature len : ',len(features))
    # #dataframe of 1 row and 1127 columns with word names :
    # d = pd.DataFrame(0, index=np.arange(len(df)), columns=features)
    # # we need to replace the index by the features name of the matrix
    # print('Dataframe of zeros : ', d)
    # print('Dataframe of zeros shape : ', d.shape)
    # print('The Dataframe of the embedded matrix : ',df)
    # print('The shape of the df matrix : ', df.shape)
    # return df

In [281]:
def predict_emdedded_matrix(doc_df):
    print('loading the MultiOutputClassifier model with torch ... ')
    loaded_model = torch.load('tfidf_model_2.pt')
    print('Predicting the document (dataframed embedded matrix) ...  ')
    print(doc_df.shape)
    pred = loaded_model.predict(doc_df)
    print('prediction : ',pred)
    doc_pred = pd.DataFrame(pred)
    # print('Dataframe of the prediction array : ',doc_pred)
    target_labels = pd.read_csv('/Users/maurelco/Developer/Python/Projet4/data/Cleaned/tfidf_labels_tags.csv')
    array_tags = []
    for row in target_labels.values:
        array_tags.append(row[0])
    ('array_tags: ',array_tags)
    # print('Type of array_tags : ',type(array_tags))
    doc_pred.columns = array_tags
    doc_pred = doc_pred.T[0]
    ('Dataframe with columns names : ', doc_pred)
    # print('Transpose Dtaframe with column names : ', doc_pred.T)
    # print('Type of the transpose : ', type(doc_pred.T[0]))
    doc_tags = doc_pred[doc_pred == 1]
    # print('Type of doc_tags : ', type(doc_tags))
    # ('Doc Tags : ', doc_tags)
    # print('Type of the Doc_Tags column : ', type(doc_tags))
    # print('array of the tags column', np.array(doc_tags.index))
    return np.array(doc_tags.index)

In [280]:
def fonction_api(title, text):
    print('1/ Preprocessing the title & Text ...')
    #call the preprocessing fonction
    text = preprocess_fct(title, text)
    # print(text)
    # print(type(text))
    print('2/ Transforming the Text into an embedding matrix ... ')
    #call the USE
    doc_df = tfidf_embedding(text)
    print('Embedded matrix Dataframe : ',doc_df)
    target_class = pd.read_csv('/Users/maurelco/Developer/Python/Projet4/data/Cleaned/tfidf_class_tags.csv')
    doc_df.columns = target_class
    # print('Embedded matrix Dataframe with column name : ',doc_df)
    # print('Is there recognized words? : ', doc_df > 0)
    print('3/ Load the MultiOutputClassifier and 126 target_labels to predict thet tags of the input text ...')
    array = predict_emdedded_matrix(doc_df)
    print('4/ print tags ...')
    for tag in array:
        print("\033[1m" + tag + "\033[0m")

In [16]:
title = 'How to create a dataframe with python?'
text = '<p> I have a big numpy array with 30 columns and 100 rows but am not sure how to use the pandas library in python to transform this array in a dataframe. Could you please help me here? <p>'

In [270]:
title_1 = 'SQL - efficient way to aggregate boolean values (postgresql)'
text_1 = " <p> Let's assume table with 3 columns (originally it's a big table): id, is_deleted, date. I have to check if given id's are deleted or not and create new column with this value (TRUE or FALSE). Let's simplify it to below table (before): id is_deleted date A False 03-07-2022 A True	04-07-2022 B False 05-07-2022 B	False 06-07-2022 C	True 07-07-2022 (after): id is_deleted	date deleted A	True 03-07-2022 TRUE A False 04-07-2022	TRUE B	False 05-07-2022 FALSE B	False	06-07-2022	FALSE C	True 07-07-2022	TRUE So we can see that row with ids A and C should have True value in new column. For given id could be more than one TRUE value in is_deleted column. If any id has at least one TRUE value, all rows with given id should be deleted (TRUE value in new column). I need to do it inside this table, without group by, cuz by choosing group by, I have to create another CTE to join it with and it complicates a problem and performance. I want to just create single column inside this table with new deleted value. I've found bool_or function, but it won't work with window functions in redshift, my code: bool_or(is_deleted) over(partition by id) as is_del I can't use max, sum functions on boolean. Casting bool to int worsens the performance. Is there any other way to do it using booleans and keep good performance? Thank you."

In [279]:
title_3 = 'Joining multiple tables in SQL'
text_3 = "Can sombody Explains me about joins? Inner join selects common data based on where condition. Left outer join selects all data from left irrespective of common but takes common data from right table and vice versa for Right outer. I know the basics but question stays when it comes to join for than 5, 8, 10 tables. Suppose I have 10 tables to join. If I have inner join with the first 5 tables and now try to apply a left join with the 6th table, now how the query will work? I mean to say now the result set of first 5 tables will be taken as left table and the 6th one will be considerded as Right table? Or only Fifth table will be considered as left and 6th as right? Please help me regarding this."

In [285]:
title_4 = 'Create a SQL table using JOIN'
text_4 = "I split the original table into 3 tables, and now I want to create a table/view that is equivalent to the original table, but I couldn't get it work at all. Below: CREATE TABLE BUSINESS_DATES AS( SELECT ttxid, dba_start_date, dba_end_date,location_start_date, location_end_date FROM BUSINESSLOCATION ORDER BY dba_start_date) WITH DATA; CREATE TABLE BUSINESS_OWNERS AS( SELECT ttxid, certificate_number, ownership_name, dba_name FROM BUSINESSLOCATION) WITH DATA; CREATE TABLE BUSINESS_INFO AS( SELECT ttxid, full_business_address, city, state, business_zip, mailing_address_1 FROM BUSINESSLOCATION) WITH DATA; The SQL command I used to combine all tables: CREATE VIEW ORIGINAL_TABLE AS ( SELECT * FROM BUSINESS_DATES AS BD LEFT JOIN BUSINESS_OWNERS AS BO ON BA.ttxid = BO.ttxid) WITH DATA;"

In [56]:
#vocaubalry within instance of TfidfVectorizer is :
# vocabulary={'__init__': 0, 'a': 1, 'absolute': 2, 'accept': 3,
#             'access': 4, 'accomplish': 5, 'according': 6,
#             'account': 7, 'achieve': 8, 'across': 9,
#             'action': 10, 'active': 11, 'activity': 12,
#             'actual': 13, 'actually': 14, 'add': 15,
#             'adding': 16, 'additional': 17, 'admin': 18,
#             'advance': 19, 'advice': 20, 'ajax': 21,
#             'alert': 22, 'algorithm': 23, 'all': 24,
#             'alloc': 25, 'allow': 26, 'allowed': 27,
#             'allows': 28, 'almost': 29, ...}

In [87]:
fonction_api(title=title, text=text)

1/ Preprocessing the title & Text ...
create dataframe python big numpy array column row sure use panda library python transform array dataframe could please help
<class 'str'>
2/ Transforming the Text into an embedding matrix ... 
printing text :  create dataframe python big numpy array column row sure use panda library python transform array dataframe could please help
printing the text transformed into an array ['create dataframe python big numpy array column row sure use panda library python transform array dataframe could please help']
printing the Instance of TFidfVectorizer which has the loaded vocabulary from "feature.pkl" :  TfidfVectorizer(decode_error='replace',
                vocabulary={'__init__': 0, 'a': 1, 'absolute': 2, 'accept': 3,
                            'access': 4, 'accomplish': 5, 'according': 6,
                            'account': 7, 'achieve': 8, 'across': 9,
                            'action': 10, 'active': 11, 'activity': 12,
                        

array([[0., 0., 0., ..., 0., 0., 0.]])

In [217]:
fonction_api(title=title, text=text)

1/ Preprocessing the title & Text ...
create dataframe python big numpy array column row sure use panda library python transform array dataframe could please help
<class 'str'>
2/ Transforming the Text into an embedding matrix ... 
Embedded matrix Dataframe :     0     1     2     3     4     5     6     7     8     9     ...  1117  \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

   1118  1119  1120  1121  1122  1123  1124  1125  1126  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[1 rows x 1127 columns]
3/ Load the MultiOutputClassifier and 126 target_labels to predict thet tags of the input text ...
loading the MultiOutputClassifier model with torch ... 
Predicting the document (dataframed embedded matrix) ...  
(1, 1127)
prediction :  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [282]:
fonction_api(title_3,text_3)

1/ Preprocessing the title & Text ...
Final text :  ['joining', 'multiple', 'table', 'sql', 'sombody', 'explains', 'join', 'inner', 'join', 'selects', 'common', 'data', 'based', 'condition', 'left', 'outer', 'join', 'selects', 'data', 'left', 'irrespective', 'common', 'take', 'common', 'data', 'right', 'table', 'vice', 'versa', 'right', 'outer', 'know', 'basic', 'question', 'stay', 'come', 'join', 'table', 'suppose', 'table', 'join', 'inner', 'join', 'first', 'table', 'try', 'apply', 'left', 'join', 'th', 'table', 'query', 'work', 'mean', 'say', 'result', 'set', 'first', 'table', 'taken', 'left', 'table', 'th', 'one', 'considerded', 'right', 'table', 'fifth', 'table', 'considered', 'left', 'th', 'right', 'please', 'help', 'regarding']
2/ Transforming the Text into an embedding matrix ... 
Embedded matrix Dataframe :     0     1     2     3     4     5     6     7     8     9     ...  1117  \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

   1118  1119  112

In [286]:
fonction_api(title_4,text_4)

1/ Preprocessing the title & Text ...
Final text :  ['create', 'sql', 'table', 'using', 'join', 'split', 'original', 'table', 'table', 'want', 'create', 'table', 'view', 'equivalent', 'original', 'table', 'get', 'work', 'create', 'table', 'business_dates', 'select', 'ttxid', 'dba_start_date', 'dba_end_date', 'location_start_date', 'location_end_date', 'businesslocation', 'order', 'dba_start_date', 'data', 'create', 'table', 'business_owners', 'select', 'ttxid', 'certificate_number', 'ownership_name', 'dba_name', 'businesslocation', 'data', 'create', 'table', 'business_info', 'select', 'ttxid', 'full_business_address', 'city', 'state', 'business_zip', 'mailing_address_', 'businesslocation', 'data', 'sql', 'command', 'used', 'combine', 'table', 'create', 'view', 'original_table', 'select', 'business_dates', 'bd', 'left', 'join', 'business_owners', 'bo', 'ba.ttxid', 'bo.ttxid', 'data']
2/ Transforming the Text into an embedding matrix ... 
Embedded matrix Dataframe :     0     1     2    

In [143]:
features = pickle.load(open("feature.pkl","rb"))
doc_df = pd.DataFrame(doc)
from collections import OrderedDict
sorted_dict = OrderedDict(sorted(features.items(), key=lambda x: x[1]))
doc_df.columns = sorted_dict
doc_df
# sorted_features = sorted(features.items(), key=lambda x: x[0])

779


,__init__,a,absolute,accept,access,accomplish,according,account,achieve,across,...,x,xaml,xcode,xml,xmlns,y,year,yes,yet,z
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# features :  {'sum': 974, 'value': 1073, 'html': 473, 'table': 988, 'using': 1065, 'loop': 582, 'javascript': 523, 'get': 428, 'updated': 1055, 'ajax': 21, 'php': 723, 'query': 781, 'time': 1017, 'entry': 332, 'total': 1028, 'row': 845, 'click': 170, 'add': 15, 'button': 136, 'although': 32, 'line': 559, 'third': 1008, 'always': 33, 'missing': 622, 'code': 177, 'input': 500, 'id': 478, 'choose': 165, 'amount': 34, 'type': 1041, 'submit': 967, 'tr': 1029, 'document': 294, 'ready': 795, 'function': 417, 'far': 376, 'var': 1074, 'val': 1069, 'url': 1060, 'datatype': 247, 'success': 968, 'append': 47, 'correctly': 225, 'getelementbyid': 429, 'length': 553, 'i++': 476, 'database': 243, 'configuration': 201, 'mysql': 640, 'host': 470, 'localhost': 571, 'root': 843, 'select': 875, 'user': 1064, 'fetch': 382, 'echo': 312, 'maybe': 607, 'guy': 449, 'best': 104, 'practice': 741, 'worked': 1106, 'asp': 66, 'net': 649, 'web': 1090, 'api': 43, 'method': 617, 'window': 1101, 'app': 44, 'thinking': 1007, 'rest': 837, 'service': 889, 'v': 1068, 'anyway': 41, 'receiving': 800, 'desktop': 269, 'client': 173, 'simply': 906, 'sends': 883, 'xml': 1120, 'json': 531, 'rather': 789, 'remote': 818, 'according': 6, 'c#': 139, 'longer': 579, 'supported': 978, 'point': 735, 'microsoft': 618, 'framework': 413, 'creating': 234, 'http': 474, 'www': 1116, 'server': 888, 'ii': 482, 'anybody': 39, 'make': 590, 'comment': 186, 'required': 830, 'disabled': 286, 'show': 901, 'set': 892, 'manually': 596, 'wanted': 1087, 'label': 537, 'adding': 16, 'theme': 1004, 'field': 384, 'empty': 321, 'checked': 163, 'p': 694, 'class': 167, 'yes': 1124, 'post': 740, 'message': 615, 'clicking': 172, 'suggestion': 973, 'behind': 102, 'swift': 981, 'upload': 1057, 'mb': 608, 'file': 386, 'making': 591, 'image': 484, 'video': 1079, 'noticed': 661, 'session': 891, 'allows': 28, 'stream': 957, 'func': 416, 'string': 958, 'int': 506, 'shared': 898, 'nil': 655, 'title': 1020, 'src': 936, 'data': 242, 'encoding': 325, 'utf': 1066, 'filename': 387, 'png': 734, 'switch': 982, 'case': 148, 'response': 836, 'everything': 339, 'failure': 375, 'small': 913, 'size': 910, 'limit': 558, 'suggested': 972, 'solved': 919, 'understand': 1049, 'passing': 709, 'collection': 179, 'object': 669, 'take': 989, 'interface': 510, 'public': 772, 'void': 1084, 'list': 562, 'call': 143, 'fails': 374, 'system': 985, 'convert': 221, 'generic': 427, 'array': 60, 'cause': 150, 'given': 432, 'obj': 668, 'pa': 695, 'anything': 40, 'finding': 393, 'place': 726, 'go': 435, 'fix': 398, 'setting': 893, 'identity': 479, 'authentication': 79, 'project': 762, 'im': 483, 'push': 775, 'log': 574, 'back': 90, 'backend': 91, 'taken': 990, 'github': 431, 'com': 181, 'implementation': 486, 'io': 515, 'en': 322, 'pretty': 745, 'much': 636, 'understanding': 1050, 'solve': 918, 'keep': 533, 'going': 437, 'mentioned': 613, 'route': 844, 'render': 822, 'provider': 770, 'store': 952, 'console': 209, 'path': 711, 'header': 459, 'callback': 144, 'state': 944, 'export': 361, 'default': 255, 'connect': 205, 'import': 489, 'j': 520, 'scope': 862, 'profile': 760, 'react': 791, 'setup': 894, 'extends': 364, 'h': 450, 'welcome': 1092, 'sample': 853, 'continue': 219, 'onclick': 676, 'this': 1009, 'redirect': 805, 'login': 578, 'never': 651, 'error': 337, 'handling': 453, 'token': 1023, 'process': 755, 'env': 333, 'production': 759, 'module': 629, 'accept': 3, 'then': 1005, 'expected': 358, 'uri': 1059, 'c': 138, 'due': 304, 'happens': 456, 'old': 674, 'version': 1078, 'library': 556, 'normally': 658, 'dependency': 263, 'python': 779, 'turn': 1038, 'big': 106, 'got': 440, 'angular': 36, 'side': 904, 'test': 999, 'content-type': 217, 'text': 1002, 'put': 776, 'thank': 1003, 'returning': 841, 'active': 11, 'directory': 284, 'attempting': 75, 'domain': 296, 'port': 738, 'valid': 1070, 'enough': 328, 'permission': 720, 'instance': 504, 'achieve': 8, 'bool': 118, 'null': 665, 'documentation': 295, 'credential': 235, 'specified': 930, 'constructor': 211, 'passed': 708, 'thrown': 1015, 'actual': 13, 'thus': 1016, 'execute': 349, 'edit': 314, 'screenshot': 864, 'shown': 903, 'calling': 145, 'password': 710, 'attempt': 74, 'validate': 1071, 'invalid': 513, 'exist': 353, 'though': 1010, 'displayed': 288, 'writing': 1113, 'node': 656, 'js': 530, 'find': 392, 'struct': 959, 'started': 942, 'learn': 549, 'snippet': 914, 'r': 784, 'wondering': 1103, 'whether': 1095, 'step': 949, 'looking': 581, 'better': 105, 'delete': 260, 'name': 641, 'selected': 876, 'learning': 550, 'basic': 97, 'sql': 935, 'hope': 469, 'ask': 63, 'sense': 884, 'request': 828, 'original': 686, 'tostring': 1027, 'db': 251, 'date': 248, 'datetime': 249, 'changed': 158, 'complete': 193, 'aspx': 67, 'language': 540, 'admin': 18, 'master': 600, 'cs': 236, 'head': 458, 'g': 420, 'linq': 560, 'ui': 1045, 'partial': 706, 'page': 698, 'protected': 766, 'sender': 881, 'alert': 22, 'cmd': 176, 'connection': 207, 'open': 679, 'reader': 793, 'read': 792, 'close': 174, 'exception': 346, 'grid': 444, 'mvc': 639, 'operation': 682, 'showing': 902, 'controller': 220, 'load': 567, 'record': 803, 'loading': 569, 'layout': 547, 'view': 1080, 'role': 842, 'placeholder': 727, 'search': 868, 'height': 460, 'width': 1099, 'px': 777, 'btn': 128, 'separate': 886, 'mobile': 623, 'center': 155, 'status': 947, 'application': 49, 'ul': 1046, 'save': 854, 'sort': 925, 'element': 319, 'asked': 64, 'sequence': 887, 'high': 466, 'range': 788, 'member': 611, 'command': 185, 'happen': 454, 'primary': 748, 'reading': 794, 'nice': 654, 'performance': 718, 'template': 996, 'several': 895, 'requirement': 831, 'solution': 917, 'basically': 98, 'display': 287, 'model': 625, 'looked': 580, 'binding': 110, 'searched': 869, 'either': 318, 'direction': 282, 'clear': 169, 'displaying': 289, 'advance': 19, 'way': 1089, 'difference': 281, 'panda': 699, 'frame': 412, 'contains': 215, 'three': 1013, 'successfully': 970, 'cell': 154, 'min': 620, 'day': 250, 'loaded': 568, 'index': 495, 'navigation': 645, 'changing': 159, 'contact': 212, 'menu': 614, 'href': 472, 'copy': 223, 'img': 485, 'jpg': 528, 'browser': 127, 'bar': 94, 'access': 4, 'device': 277, 'java': 522, 'i': 475, 'target': 992, 'phone': 722, 'allow': 26, 'linux': 561, 'platform': 729, 'android': 35, 'around': 59, 'aws': 87, 'virtual': 1081, 'website': 1091, 'great': 442, 'testing': 1001, 'safari': 851, 'reason': 797, 'support': 977, 'built': 134, 'storage': 951, 'future': 419, 'seem': 873, 'coming': 184, 'dont': 297, 'play': 730, 'action': 10, 'medium': 610, 'exactly': 342, 'core': 224, 'assume': 71, 'pull': 773, 'thread': 1012, 'facing': 370, 'activity': 12, 'fill': 388, 'static': 946, 'drop': 303, 'visible': 1082, 'scenario': 859, 'slow': 912, 'handler': 452, 'properly': 764, 'come': 183, 'async': 73, 'task': 993, 'allowed': 27, 'bundle': 135, 'standard': 940, 'arraylist': 61, 'updating': 1056, 'd': 241, 'e': 308, 'printstacktrace': 750, 'start': 941, 'private': 751, 'boolean': 119, 'all': 24, 'refresh': 809, 'suggest': 971, 'ip': 517, 'supposed': 980, 'based': 96, 'network': 650, 'google': 439, 'good': 438, 'approach': 53, 'b': 89, 'a': 1, 't': 986, 'm': 585, 'layer': 546, 'therefore': 1006, 'appear': 45, 'local': 570, 'protocol': 767, 'within': 1102, 'almost': 29, 'said': 852, 'apps': 55, 'existing': 354, 'matter': 604, 'lot': 583, 'term': 997, 'background': 92, 'large': 542, 'feature': 380, 'single': 907, 'appropriate': 54, 'moving': 634, 'developer': 274, 'customer': 240, 'lang': 539, 'remove': 819, 'param': 701, 'initial': 497, 'internal': 511, 'ref': 806, 'count': 227, 'goal': 436, 'obviously': 670, 'upon': 1058, 'avoid': 84, 'executed': 350, 'configured': 203, 'foo': 406, 'timeout': 1018, 'security': 872, 'article': 62, 'easily': 310, 'feel': 381, 'bad': 93, 'real': 796, 'full': 415, 'building': 133, 'followed': 403, 'tutorial': 1039, 'managed': 593, 'hello': 461, 'world': 1108, 'undefined': 1048, 'reference': 807, 'xcode': 1119, 'eclipse': 313, 'bootstrap': 121, 'dialog': 279, 'apache': 42, 'pdf': 713, 'output': 691, 'defined': 257, 'stylesheet': 964, 'xmlns': 1121, 'w': 1085, 'transform': 1034, 'format': 410, 'match': 601, 'block': 114, 'fixed': 399, 'wrap': 1110, 'desired': 268, 'form': 409, 'attribute': 77, 'entity': 331, 'visual': 1083, 'studio': 962, 'specifically': 929, 'download': 299, 'available': 83, 'bit': 111, 'installing': 503, 'k': 532, 'little': 565, 'unfortunately': 1051, 'forward': 411, 'release': 816, 'word': 1104, 'next': 653, 'insert': 501, 'tab': 987, 'jquery': 529, 'key': 534, 'auto': 81, 'trigger': 1036, 'attr': 76, 'include': 491, 'express': 362, 'parse': 704, 'auth': 78, 'picture': 724, 'whatever': 1093, 'sdk': 867, 'hit': 467, 'logged': 575, 'appreciated': 52, 'pattern': 712, 'wpf': 1109, 'various': 1075, 'definition': 258, 'normal': 657, 'particular': 707, 'follow': 402, 'c++': 140, 'compiler': 192, 'year': 1123, 'development': 276, 'downloaded': 300, 'site': 908, 'press': 744, 'product': 758, 'explain': 360, 's': 850, 'cd': 153, 'software': 916, 'seen': 874, 'map': 597, 'de': 252, 'perfectly': 716, 'completely': 195, 'latest': 544, 'delegate': 259, 'force': 407, 'ssl': 937, 'stuck': 961, 'nothing': 659, 'stuff': 963, 'useful': 1063, 'received': 799, 'report': 825, 'proxy': 771, 'socket': 915, 'crash': 231, 'tested': 1000, 'iphone': 518, 'reset': 834, 'might': 619, 'pointer': 736, 'important': 490, 'apply': 50, 'problem': 753, 'statement': 945, 'proper': 763, 'memory': 612, 'exact': 341, 'animation': 37, 'sent': 885, 'giving': 433, 'duplicate': 305, 'similar': 905, 'provide': 768, 'extra': 367, 'someone': 921, 'greatly': 443, 'appreciate': 51, 'unable': 1047, 'verify': 1077, 'django': 290, 'send': 880, 'detail': 270, 'location': 573, 'doc': 292, 'validation': 1072, 'syntax': 984, 'authorization': 80, 'relevant': 817, 'replace': 824, 'body': 117, 'notification': 662, 'removed': 820, 'z': 1126, 'l': 536, 'rel': 813, 'self': 879, 'git': 430, 'modified': 627, 'outside': 692, 'job': 526, 'including': 494, 'made': 588, 'use': 1062, 'rule': 847, 'developing': 275, 'build': 131, 'bin': 107, 'exe': 347, 'lib': 555, 'o': 667, 'manager': 595, 'occurs': 672, 'easy': 311, 'multiple': 637, 'effect': 316, 'endpoint': 326, 'creates': 233, 'share': 897, 'across': 9, 'must': 638, 'trouble': 1037, 'args': 57, 'println': 749, 'extract': 368, 'determine': 272, 'matrix': 603, 'binary': 108, 'design': 267, 'happening': 455, 'written': 1114, 'main': 589, 'std': 948, 'exit': 356, 'usage': 1061, 'argument': 58, 'expect': 357, 'len': 552, 'wait': 1086, 'f': 369, 'raw': 790, 'behavior': 100, 'thought': 1011, 'implemented': 487, 'finally': 391, 'css': 237, 'asset': 69, 'margin': 599, 'padding': 697, 'border': 122, 'example': 343, 'actually': 14, 'somewhere': 923, 'cant': 147, 'tell': 994, 'resolve': 835, 'per': 715, 'meta': 616, 'charset': 162, 'libs': 557, 'fail': 373, 'machine': 587, 'dll': 291, 'moment': 630, 'recently': 802, 'guide': 448, 'figure': 385, 'functionality': 418, 'couple': 228, 'probably': 752, 'create': 232, 'base': 95, 'params': 702, 'email': 320, 'combination': 182, 'new': 652, 'notice': 660, 'require': 829, 'handle': 451, 'level': 554, 'finished': 395, 'directly': 283, 'txt': 1040, 'final': 390, 'super': 976, 'certificate': 157, 'provided': 769, 'sun': 975, 'spring': 934, 'exec': 348, 'servlet': 890, 'filter': 389, 'invoke': 514, 'tomcat': 1024, 'util': 1067, 'run': 848, 'worker': 1107, 'processing': 756, 'boot': 120, 'javax': 524, 'timestamp': 1019, 'returned': 840, 'assuming': 72, 'completed': 194, 'le': 548, 'package': 696, 'font': 405, 'kind': 535, 'relative': 815, 'enabled': 324, 'selection': 877, 'retrieve': 839, 'depending': 264, 'guess': 446, 'consider': 208, 'later': 543, 'repository': 826, 'annotation': 38, 'related': 814, 'symbol': 983, 'wrote': 1115, 'ok': 673, 'result': 838, 'together': 1022, 'implementing': 488, 'tree': 1035, 'checking': 164, 'hold': 468, 'begin': 99, 'expression': 363, 'exists': 355, 'dynamic': 306, 'scale': 858, 'screen': 863, 'constraint': 210, 'closed': 175, 'enter': 329, 'break': 126, 'experience': 359, 'out': 690, 'sorry': 924, 'ex': 340, 'rail': 785, 'present': 743, 'content': 216, 'specific': 928, 'hi': 462, 'stack': 938, 'overflow': 693, 'live': 566, 'on': 675, 'listen': 563, 'terminal': 998, 'connected': 206, 'firefox': 396, 'chrome': 166, 'tool': 1025, 'yet': 1125, 'box': 125, 'manage': 592, 'mapping': 598, 'join': 527, 'trace': 1030, 'previous': 747, 'runtime': 849, 'namespace': 643, 'models': 626, 'character': 161, 'generated': 425, 'define': 256, 'x': 1117, 'demo': 262, 'parent': 703, 'mean': 609, 'otherwise': 689, 'color': 180, 'dynamically': 307, 'somehow': 920, 'unique': 1052, 'foreach': 408, 'logic': 577, 'typeof': 1043, 'structure': 960, 'follows': 404, 'account': 7, '__init__': 0, 'py': 778, 'installed': 502, 'double': 298, 'register': 811, 'corresponding': 226, 'produce': 757, 'condition': 198, 'ubuntu': 1044, 'azure': 88, 'starting': 943, 'listener': 564, 'environment': 334, 'ioexception': 516, 'sending': 882, 'byte': 137, 'buffer': 129, 'throw': 1014, 'oracle': 684, 'jdbc': 525, 'driver': 302, 'maven': 605, 'course': 229, 'write': 1112, 'suppose': 979, 'detect': 271, 'compare': 190, 'certain': 156, 'advice': 20, 'panel': 700, 'hour': 471, 'equivalent': 336, 'causing': 152, 'disable': 285, 'pop': 737, 'draw': 301, 'one': 677, 'except': 345, 'queue': 782, 'generating': 426, 'await': 85, 'modify': 628, 'dataframe': 244, 'temp': 995, 'global': 434, 'intent': 508, 'col': 178, 'piece': 725, 'numpy': 666, 'np': 663, 'cursor': 239, 'management': 594, 'efficient': 317, 'complex': 196, 'item': 519, 'typeerror': 1042, 'removing': 821, 'bottom': 123, 'editor': 315, 'bound': 124, 'mouse': 632, 'plugin': 732, 'purpose': 774, 'prevent': 746, 'event': 338, 'char': 160, 'area': 56, 'included': 492, 'includes': 493, 'special': 927, 'ignore': 481, 'whole': 1097, 'scroll': 866, 'widget': 1098, 'schemas': 861, 'topic': 1026, 'recent': 801, 'general': 423, 'prefer': 742, 'subject': 966, 'gcc': 422, 'online': 678, 'float': 401, 'bug': 130, 'stored': 953, 'doctype': 293, 'section': 871, 'max': 606, 'configure': 202, 'came': 146, 'req': 827, 'res': 833, 'shell': 899, 'xaml': 1118, 'opened': 680, 'finish': 394, 'org': 685, 'enable': 323, 'mode': 624, 'opening': 681, 'internet': 512, 'along': 30, 'perform': 717, 'config': 200, 'factory': 372, 'stackoverflow': 939, 'rendering': 823, 'q': 780, 'cpu': 230, 'believe': 103, 'receive': 798, 'math': 602, 'extension': 365, 'situation': 909, 'selector': 878, 'alternative': 31, 'todo': 1021, 'dev': 273, 'str': 955, 'common': 188, 'computer': 197, 'generate': 424, 'contain': 213, 'jar': 521, 'plugins': 733, 'clean': 168, 'caused': 151, 'init': 496, 'converting': 222, 'graph': 441, 'appears': 46, 'split': 933, 'ie': 480, 'logging': 576, 'laravel': 541, 'speed': 932, 'sometimes': 922, 'native': 644, 'description': 266, 'necessary': 646, 'excel': 344, 'requires': 832, 'ff': 383, 'cache': 141, 'df': 278, 'vector': 1076, 'occurred': 671, 'apple': 48, 'compiled': 191, 'property': 765, 'confused': 204, 'unit': 1053, 'aware': 86, 'white': 1096, 'fact': 371, 'hibernate': 463, 'schema': 860, 'group': 445, 'ruby': 846, 'specify': 931, 'quite': 783, 'move': 633, 'warning': 1088, 'mac': 586, 'instruction': 505, 'flag': 400, 'procedure': 754, 'asking': 65, 'script': 865, 'position': 739, 'taking': 991, 'engine': 327, 'category': 149, 'hide': 465, 'people': 714, 'random': 787, 'saying': 857, 'decided': 254, 'parsing': 705, 'additional': 17, 'builder': 132, 'context': 218, 'named': 642, 'free': 414, 'company': 189, 'behaviour': 101, 'fast': 377, 'absolute': 2, 'plot': 731, 'shape': 896, 'dictionary': 280, 'storing': 954, 'person': 721, 'short': 900, 'win': 1100, 'gui': 447, 'debugging': 253, 'operator': 683, 'strange': 956, 'successful': 969, 'game': 421, 'integer': 507, 'external': 366, 'blog': 115, 'searching': 870, 'space': 926, 'perhaps': 719, 'automatically': 82, 'blue': 116, 'icon': 477, 'alloc': 25, 'transaction': 1033, 'containing': 214, 'inner': 499, 'fatal': 379, 'algorithm': 23, 'entire': 330, 'luck': 584, 'needed': 647, 'initialize': 498, 'red': 804, 'calculate': 142, 'assign': 70, 'easier': 309, 'csv': 238, 'dataset': 245, 'saved': 855, 'saving': 856, 'equal': 335, 'track': 1032, 'deleted': 261, 'month': 631, 'nested': 648, 'interested': 509, 'ran': 786, 'y': 1122, 'stop': 950, 'launch': 545, 'least': 551, 'minute': 621, 'whenever': 1094, 'msg': 635, 'conf': 199, 'bind': 109, 'locally': 572, 'deploy': 265, 'os': 687, 'commit': 187, 'plain': 728, 'executing': 351, 'datasource': 246, 'clicked': 171, 'sleep': 911, 'others': 688, 'hidden': 464, 'accomplish': 5, 'black': 112, 'unknown': 1054, 'programming': 761, 'npm': 664, 'traceback': 1031, 'fit': 397, 'blank': 113, 'work': 1105, 'sub': 965, 'regular': 812, 'assembly': 68, 'wrapper': 1111, 'execution': 352, 'regarding': 810, 'hard': 457, 'reflect': 808, 'lambda': 538, 'faster': 378}